<a href="https://colab.research.google.com/github/Andrzej94-ai/kaggle/blob/main/kaagle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import bibliotek

In [1]:
import pandas as pd

Analiza zbioru

In [4]:
data = pd.read_csv("/content/AmesHousing.csv")

In [13]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Order,2930.0,1.465500e+03,8.459625e+02,1.0,7.332500e+02,1465.5,2.197750e+03,2.930000e+03
PID,2930.0,7.144645e+08,1.887308e+08,526301100.0,5.284770e+08,535453620.0,9.071811e+08,1.007100e+09
MS SubClass,2930.0,5.738737e+01,4.263802e+01,20.0,2.000000e+01,50.0,7.000000e+01,1.900000e+02
Lot Frontage,2440.0,6.922459e+01,2.336533e+01,21.0,5.800000e+01,68.0,8.000000e+01,3.130000e+02
Lot Area,2930.0,1.014792e+04,7.880018e+03,1300.0,7.440250e+03,9436.5,1.155525e+04,2.152450e+05
Overall Qual,2930.0,6.094881e+00,1.411026e+00,1.0,5.000000e+00,6.0,7.000000e+00,1.000000e+01
Overall Cond,2930.0,5.563140e+00,1.111537e+00,1.0,5.000000e+00,5.0,6.000000e+00,9.000000e+00
Year Built,2930.0,1.971356e+03,3.024536e+01,1872.0,1.954000e+03,1973.0,2.001000e+03,2.010000e+03
Year Remod/Add,2930.0,1.984267e+03,2.086029e+01,1950.0,1.965000e+03,1993.0,2.004000e+03,2.010000e+03
Mas Vnr Area,2907.0,1.018968e+02,1.791126e+02,0.0,0.000000e+00,0.0,1.640000e+02,1.600000e+03


In [16]:
data.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
      

In [25]:
data.select_dtypes('object').columns

Index(['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour',
       'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature',
       'Sale Type', 'Sale Condition'],
      dtype='object')

In [27]:
data.groupby('MS Zoning')['MS Zoning'].count()

MS Zoning
A (agr)       2
C (all)      25
FV          139
I (all)       2
RH           27
RL         2273
RM          462
Name: MS Zoning, dtype: int64

In [28]:
data.groupby('Street')['Street'].count()

Street
Grvl      12
Pave    2918
Name: Street, dtype: int64

In [29]:
data.groupby('Alley')['Alley'].count()

Alley
Grvl    120
Pave     78
Name: Alley, dtype: int64

In [30]:
data.groupby('Lot Shape')['Lot Shape'].count()

Lot Shape
IR1     979
IR2      76
IR3      16
Reg    1859
Name: Lot Shape, dtype: int64

In [31]:
data.groupby('Neighborhood')['Neighborhood'].count() #label_encoder

Neighborhood
Blmngtn     28
Blueste     10
BrDale      30
BrkSide    108
ClearCr     44
CollgCr    267
Crawfor    103
Edwards    194
Gilbert    165
Greens       8
GrnHill      2
IDOTRR      93
Landmrk      1
MeadowV     37
Mitchel    114
NAmes      443
NPkVill     23
NWAmes     131
NoRidge     71
NridgHt    166
OldTown    239
SWISU       48
Sawyer     151
SawyerW    125
Somerst    182
StoneBr     51
Timber      72
Veenker     24
Name: Neighborhood, dtype: int64

In [32]:
data.groupby('Land Contour')['Land Contour'].count()

Land Contour
Bnk     117
HLS     120
Low      60
Lvl    2633
Name: Land Contour, dtype: int64

In [34]:
data.groupby('House Style')['House Style'].count()

House Style
1.5Fin     314
1.5Unf      19
1Story    1481
2.5Fin       8
2.5Unf      24
2Story     873
SFoyer      83
SLvl       128
Name: House Style, dtype: int64

In [35]:
data.groupby('Fence')['Fence'].count()

Fence
GdPrv    118
GdWo     112
MnPrv    330
MnWw      12
Name: Fence, dtype: int64

In [36]:
data.groupby('Sale Type')['Sale Type'].count()

Sale Type
COD        87
CWD        12
Con         5
ConLD      26
ConLI       9
ConLw       8
New       239
Oth         7
VWD         1
WD       2536
Name: Sale Type, dtype: int64

In [37]:
data.groupby('Sale Condition')['Sale Condition'].count()

Sale Condition
Abnorml     190
AdjLand      12
Alloca       24
Family       46
Normal     2413
Partial     245
Name: Sale Condition, dtype: int64

In [45]:
data.select_dtypes(exclude = ['object', 'int64', 'float64']).columns

Index([], dtype='object')

In [47]:
data.select_dtypes(include = ['int64']).columns

Index(['Order', 'PID', 'MS SubClass', 'Lot Area', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd',
       'Fireplaces', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
       '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold',
       'Yr Sold', 'SalePrice'],
      dtype='object')

In [60]:
data.select_dtypes(include = ['float64']).columns

Index(['Lot Frontage', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath',
       'Garage Yr Blt', 'Garage Cars', 'Garage Area'],
      dtype='object')

In [55]:
#plik treningowy - wszystko poza 2010 rokiem
#plik testowy - sprzedażaż w 2010
# train = pd.read_csv('/content/train.csv')
# test = pd.read_csv('/content/test.csv')
# target = pd.read_csv('/content/target.csv')

In [73]:
train = data[data['Yr Sold'] != 2010]
train.reset_index(drop=True, inplace = True)
test = data[data['Yr Sold'] == 2010]
test.reset_index(drop=True, inplace = True)

In [76]:
train['SalePrice'].describe()

count      2589.000000
mean     181875.888374
std       80504.708260
min       13100.000000
25%      129900.000000
50%      161500.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64